In [1]:
import os
import pandas as pd
import numpy as np
!pip install surprise
#!pip install scikit-surprise # if the first line does not work

# Create Dataframe

In [2]:
# load data
df = pd.read_csv('./data/final_amazon_beauty.csv')
data = df[['rating', 'reviewer_id', 'product_id']]

df.head(3)

/Users/bea/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,rating,verified,review_time,reviewer_id,product_id,reviewer_name,reviewer_text,summary,vote,title,brand,rank,main_cat,description,also_view,also_buy,price,similar_item,details
0,5,False,2000-06-03,A2XMFX1BR0IJFJ,0061073717,Jonathan Reed (jonathan.reed2@virgin.net),This calender is brilliant and has plenty of g...,Futurama rules,2,Workout Headphones by Arena Essentials,HarperCollins,"3,235,148inBeautyamp;PersonalCare(",All Beauty,NaN,NaN,NaN,NaN,NaN,NaN
1,5,False,2000-05-06,ATKPYXA8XFKGJ,0061073717,Gwen Bates,This calender really is great. In addition to...,A great gift for any futurama fan!,NaN,Workout Headphones by Arena Essentials,HarperCollins,"3,235,148inBeautyamp;PersonalCare(",All Beauty,NaN,NaN,NaN,NaN,NaN,NaN
2,1,True,2015-02-19,A1V6B6TNIC10QE,0143026860,theodore j bigham,great,One Star,NaN,Black Diamond,Swedish Beauty,"1,462,563inBeautyamp;PersonalCare(",All Beauty,['This is the NEW PURPLE BOTTLE that just came...,['B000LXTNMW'],NaN,NaN,NaN,NaN


# Using SVD using Surprise

In [3]:
# from surprise import Reader, Dataset, SVD, evaluate

# Following Surprise documentation examples 
# https://surprise.readthedocs.io/en/stable/getting_started.html

from surprise import Reader, Dataset, SVD, NormalPredictor
from collections import defaultdict


from surprise import Dataset
from surprise.model_selection import cross_validate


# As we're loading a custom dataset, we need to define a reader.
reader = Reader(rating_scale=(0.5, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(data[['reviewer_id', 'product_id', 'rating']], reader)

# We'll use the famous SVD algorithm.
algo = SVD()

# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2580  1.2541  1.2544  1.2517  1.2561  1.2548  0.0021  
MAE (testset)     0.9718  0.9698  0.9711  0.9704  0.9728  0.9712  0.0011  
Fit time          20.97   23.95   25.14   22.77   21.36   22.84   1.56    
Test time         0.65    0.62    0.70    0.62    0.62    0.64    0.03    


{'test_rmse': array([1.25795442, 1.25407772, 1.25437031, 1.25167785, 1.2560563 ]),
 'test_mae': array([0.97182642, 0.9698253 , 0.97109358, 0.97035546, 0.9728277 ]),
 'fit_time': (20.968918800354004,
  23.94983196258545,
  25.135207891464233,
  22.771068811416626,
  21.35887598991394),
 'test_time': (0.6536121368408203,
  0.6189765930175781,
  0.7026939392089844,
  0.6185710430145264,
  0.6182332038879395)}

In [4]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

1.2924020668977898
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


In [ ]:
# We can now use the algorithm that yields the best rmse:
algo = gs.best_estimator['rmse']
trainset = data.build_full_trainset()
algo.fit(trainset)

In [ ]:
algo.predict(621,1)

In [ ]:
df_data = df[['reviewer_id', 'product_id', 'rating', 'title' ]]
df_data[df_data['reviewer_id']=='A2GJX2KCUSR0EI'].sort_values(by = 'rating',ascending=False)[:10]

In [ ]:
# From Surprise documentation: https://surprise.readthedocs.io/en/stable/FAQ.html
def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [ ]:
# Predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

In [ ]:
top_n = get_top_n(predictions, n=10)

In [ ]:
top_n.items()

In [ ]:
# Print the recommended items for all user 1
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])
    if uid == 1:
        break

In [15]:
test_reviewer_id = "ATKPYXA8XFKGJ"
df_data[df_data['reviewer_id']==test_reviewer_id].sort_values(by = 'rating',ascending=False)[:20]

,reviewer_id,product_id,rating,title,brand
2,ATKPYXA8XFKGJ,0061073717,5,Workout Headphones by Arena Essentials,HarperCollins
3,ATKPYXA8XFKGJ,0061073717,5,Workout Headphones by Arena Essentials,HarperCollins
